In [1]:
#! ./install_local_setup.sh
! cp ../reynolds_summa_setup_fresh/settings/snow_zLocalParamInfo.txt ./settings/snow_zLocalParamInfo.txt
! cp ../reynolds_summa_setup_fresh/settings/snow_zDecisions.txt ./settings/snow_zDecisions.txt
! cp ../reynolds_summa_setup_fresh/settings/snow_zOutputControl.txt ./settings/snow_zOutputControl.txt

In [2]:
%pylab inline
import pandas as pd
import pysumma as ps
import xarray as xr
from matplotlib import cm
import seaborn as sns
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
sns.set_context('talk')
mpl.style.use('seaborn-bright')
mpl.rcParams['figure.figsize'] = (18, 12)

Populating the interactive namespace from numpy and matplotlib


In [3]:
def vapor_pressure(spechum, airpres):
    eps = 0.62196351
    pres = airpres / 1000
    return 10 * ((spechum * pres) / (spechum + eps - eps * pres))

def longwave(temp, vp):
    emis = 1 - (1 + (46.5 * vp / temp)) * np.exp(
            -np.sqrt((1.2 + 3. * (46.5 * vp / temp))))
    stefan = 5.669e-8
    return emis * stefan * np.power(temp, 4)

def validate_layer_params(params):
    for i in range(1,5):
        assert params[f'zmaxLayer{i}_upper'] <= params[f'zmaxLayer{i}_lower'], \
               "{} - {} - {}".format(i, params[f'zmaxLayer{i}_upper'], params[f'zmaxLayer{i}_lower'])
        assert params[f'zmaxLayer{i}_upper'] / params[f'zminLayer{i}'] >= 2.5, \
               "{} - {} - {}".format(i, params[f'zmaxLayer{i}_upper'], params[f'zminLayer{i}'])
        assert params[f'zmaxLayer{i}_upper'] / params[f'zminLayer{i+1}'] >= 2.5, \
               "{} - {} - {}".format(i, params[f'zmaxLayer{i}_upper'], params[f'zminLayer{i+1}'])

In [4]:

dt_list = ['-2.0', '+0.0', '+2.0', '+4.0']

In [5]:
ds = xr.open_dataset('./forcing/forcing_above_aspen.nc')

In [6]:
default_params = {'zminLayer1':      0.0075,   'zminLayer2':       0.0100,  'zminLayer3':       0.0500,  'zminLayer4':       0.1000,  'zminLayer5': 0.2500, 
                  'zmaxLayer1_lower': 0.0500,  'zmaxLayer2_lower': 0.2000,  'zmaxLayer3_lower': 0.5000,  'zmaxLayer4_lower': 1.0000, 
                  'zmaxLayer1_upper': 0.0300,  'zmaxLayer2_upper': 0.1500,  'zmaxLayer3_upper': 0.3000,  'zmaxLayer4_upper': 0.7500 }

one_layer     = {'zminLayer1':       0.0075, 'zminLayer2':       1000.0, 'zminLayer3':       1000., 'zminLayer4':       1000., 'zminLayer5': 1000.,
                 'zmaxLayer1_lower': 9000.0, 'zmaxLayer2_lower': 9000.0, 'zmaxLayer3_lower': 9000., 'zmaxLayer4_lower': 9000., 
                 'zmaxLayer1_upper': 3000.0, 'zmaxLayer2_upper': 9000.0, 'zmaxLayer3_upper': 9000., 'zmaxLayer4_upper': 9000. } 

thick_2_layer = {'zminLayer1':       0.0075, 'zminLayer2':       0.0100, 'zminLayer3':        100., 'zminLayer4':      100., 'zminLayer5': 100.,
                 'zmaxLayer1_lower': 0.1250, 'zmaxLayer2_lower': 9000.0, 'zmaxLayer3_lower': 9000., 'zmaxLayer4_lower': 9000., 
                 'zmaxLayer1_upper': 0.1250, 'zmaxLayer2_upper': 9000.0, 'zmaxLayer3_upper': 9000., 'zmaxLayer4_upper': 9000. } 

mid_2_layer = {'zminLayer1':       0.0075, 'zminLayer2':       0.0100, 'zminLayer3':        100., 'zminLayer4':      100., 'zminLayer5': 100.,
               'zmaxLayer1_lower': 0.050, 'zmaxLayer2_lower': 9000.0, 'zmaxLayer3_lower': 9000., 'zmaxLayer4_lower': 9000., 
               'zmaxLayer1_upper': 0.050, 'zmaxLayer2_upper': 9000.0, 'zmaxLayer3_upper': 9000., 'zmaxLayer4_upper': 9000. } 

thin_2_layer = {'zminLayer1':       0.0075, 'zminLayer2':       0.0100, 'zminLayer3':        100., 'zminLayer4':      100., 'zminLayer5': 100.,
                'zmaxLayer1_lower': 0.0250, 'zmaxLayer2_lower': 9000.0, 'zmaxLayer3_lower': 9000., 'zmaxLayer4_lower': 9000., 
                'zmaxLayer1_upper': 0.0250, 'zmaxLayer2_upper': 9000.0, 'zmaxLayer3_upper': 9000., 'zmaxLayer4_upper': 9000. } 

thick_3_layer = {'zminLayer1': 0.01, 'zminLayer2': 0.01, 'zminLayer3': 0.01, 'zminLayer4': 1000., 'zminLayer5': 1000.,
               'zmaxLayer1_lower': 0.125, 'zmaxLayer2_lower': 0.2, 'zmaxLayer3_lower': 9000., 'zmaxLayer4_lower': 9000.,
               'zmaxLayer1_upper': 0.125, 'zmaxLayer2_upper': 0.2, 'zmaxLayer3_upper': 9000., 'zmaxLayer4_upper': 9000.}
    
mid_3_layer = {'zminLayer1': 0.01, 'zminLayer2': 0.01, 'zminLayer3': 0.01, 'zminLayer4': 1000., 'zminLayer5': 1000.,
               'zmaxLayer1_lower': 0.050, 'zmaxLayer2_lower': 0.2, 'zmaxLayer3_lower': 9000., 'zmaxLayer4_lower': 9000.,
               'zmaxLayer1_upper': 0.050, 'zmaxLayer2_upper': 0.2, 'zmaxLayer3_upper': 9000., 'zmaxLayer4_upper': 9000.}

thin_3_layer = {'zminLayer1': 0.01, 'zminLayer2': 0.01, 'zminLayer3': 0.01, 'zminLayer4': 1000., 'zminLayer5': 1000.,
               'zmaxLayer1_lower': 0.025, 'zmaxLayer2_lower': 0.2, 'zmaxLayer3_lower': 9000., 'zmaxLayer4_lower': 9000.,
               'zmaxLayer1_upper': 0.025, 'zmaxLayer2_upper': 0.2, 'zmaxLayer3_upper': 9000., 'zmaxLayer4_upper': 9000.}

thick_4_layer = {'zminLayer1': 0.01, 'zminLayer2': 0.01, 'zminLayer3': 0.01, 'zminLayer4': 0.015, 'zminLayer5': 1000.,
               'zmaxLayer1_lower': 0.075, 'zmaxLayer2_lower': 0.15, 'zmaxLayer3_lower': 0.20, 'zmaxLayer4_lower': 9000.,
               'zmaxLayer1_upper': 0.075, 'zmaxLayer2_upper': 0.15, 'zmaxLayer3_upper': 0.20, 'zmaxLayer4_upper': 9000.}
    
mid_4_layer = {'zminLayer1': 0.01, 'zminLayer2': 0.01, 'zminLayer3': 0.01, 'zminLayer4': 0.015, 'zminLayer5': 1000.,
               'zmaxLayer1_lower': 0.050, 'zmaxLayer2_lower': 0.125, 'zmaxLayer3_lower': 0.20, 'zmaxLayer4_lower': 9000.,
               'zmaxLayer1_upper': 0.050, 'zmaxLayer2_upper': 0.125, 'zmaxLayer3_upper': 0.20, 'zmaxLayer4_upper': 9000.}

thin_4_layer = {'zminLayer1': 0.01, 'zminLayer2': 0.01, 'zminLayer3': 0.01, 'zminLayer4': 0.015, 'zminLayer5': 1000.,
               'zmaxLayer1_lower': 0.025, 'zmaxLayer2_lower': 0.100, 'zmaxLayer3_lower': 0.20, 'zmaxLayer4_lower': 9000.,
               'zmaxLayer1_upper': 0.025, 'zmaxLayer2_upper': 0.100, 'zmaxLayer3_upper': 0.20, 'zmaxLayer4_upper': 9000.}

for i, l in enumerate([one_layer, thin_2_layer, thick_2_layer, mid_2_layer, 
                       thick_3_layer, mid_3_layer, thin_3_layer,
                       thick_4_layer, mid_4_layer, thin_4_layer]):
    print(i)
    validate_layer_params(l)
    

config_all = {'{}K_CLM'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                     'decisions': {'snowLayers': 'CLM_2010'},
                                     'parameters': default_params} for dt in dt_list}

config_1layer = {'{}K_1L_all'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                            'decisions': {'snowLayers': 'CLM_2010'},
                                            'parameters': one_layer} for dt in dt_list}

config_2layer = {'{}K_2L_thin'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                            'decisions': {'snowLayers': 'CLM_2010'},
                                            'parameters': thin_2_layer} for dt in dt_list}
config_2layer.update({'{}K_2L_mid'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                                 'decisions': {'snowLayers': 'CLM_2010'},
                                                 'parameters': mid_2_layer} for dt in dt_list})
config_2layer.update({'{}K_2L_thick'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                                   'decisions': {'snowLayers': 'CLM_2010'},
                                                   'parameters': thick_2_layer} for dt in dt_list})

config_3layer = {'{}K_3L_thin'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                            'decisions': {'snowLayers': 'CLM_2010'},
                                            'parameters': thin_3_layer} for dt in dt_list}
config_3layer.update({'{}K_3L_mid'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                                 'decisions': {'snowLayers': 'CLM_2010'},
                                                 'parameters': mid_3_layer} for dt in dt_list})
config_3layer.update({'{}K_3L_thick'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                                   'decisions': {'snowLayers': 'CLM_2010'},
                                                   'parameters': thick_3_layer} for dt in dt_list})

config_4layer = {'{}K_4L_thin'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                            'decisions': {'snowLayers': 'CLM_2010'},
                                            'parameters': thin_4_layer} for dt in dt_list}
config_4layer.update({'{}K_4L_mid'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                                 'decisions': {'snowLayers': 'CLM_2010'},
                                                 'parameters': mid_4_layer} for dt in dt_list})
config_4layer.update({'{}K_4L_thick'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                                   'decisions': {'snowLayers': 'CLM_2010'},
                                                   'parameters': thick_4_layer} for dt in dt_list})

config_jrdn = {'{}K_JRDN'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                       'decisions': {'snowLayers': 'jrdn1991'},
                                       'parameters': default_params} for dt in dt_list}

0
1
2
3
4
5
6
7
8
9


In [7]:
config_all.update(config_jrdn)
config_all.update(config_1layer)
config_all.update(config_2layer)
config_all.update(config_3layer)
config_all.update(config_4layer)

In [8]:
#failed = ['+0.0K_JRDN']
#config_all = {k: config_all[k] for k in failed}

In [9]:
len(list(config_all.keys()))

48

In [10]:
executable = '/pool0/data/andrbenn/summa/bin/summa.exe'

In [11]:
ens_dt = ps.Ensemble(executable, config_all, num_workers=24)

In [12]:
ens_dt.run('local')

In [13]:
print(ens_dt.summary()['error'])

['+2.0K_3L_thin']


In [16]:
ens_dt.rerun_failed('local')

In [17]:
print(ens_dt.simulations['+2.0K_3L_thin'].stdout[-1000:])

 0
2008  9 18  0  0
2008  9 19  0  0
2008  9 20  0  0
2008  9 21  0  0
2008  9 22  0  0
2008  9 23  0  0
2008  9 24  0  0
2008  9 25  0  0
2008  9 26  0  0
2008  9 27  0  0
2008  9 28  0  0
2008  9 29  0  0
2008  9 30  0  0

initial date/time = 2020-06-09  16:02:24.137
  final date/time = 2020-06-09  16:07:22.105

     elapsed init =   3.0000000E-02 s
    fraction init =   1.0068195E-04

    elapsed setup =   3.2000000E-02 s
   fraction setup =   1.0739408E-04

  elapsed restart =   9.0000000E-03 s
 fraction restart =   3.0204586E-05

     elapsed read =    17.84300     s
    fraction read =   5.9882269E-02

    elapsed write =    118.5060     s
   fraction write =   0.3977138    

  elapsed physics =    161.1340     s
 fraction physics =   0.5407762    

     elapsed time =    297.9680     s
       or              4.966133     m
       or             8.2768889E-02 h
       or             3.4487037E-03 d

   number threads =          1

 FORTRAN STOP: finished simulation successfully.


In [18]:
print(ens_dt.simulations['+2.0K_3L_thin'].stderr[-1000:])

In [ ]:
print(ens_dt.simulations['+0.0K_JRDN'].stderr[-1000:])